In [53]:
import math
import numpy as np

In [5]:
import requests

# reads an uri and returns its content split by line, deals with \n and \r
def read_uri(uri):
  response = requests.get(uri)
  content = response.text.strip().replace("\r", "").split("\n")
  return content

In [1]:
def dp_change(money, coins):
  # [0  1  2  3  4  5  6  7  8  9 10]
  # [1 0 0 0 0 0 0 0 0 0 0]
  # [0 1 0 1 0 0 0 0 0 0 0]
  # [0 0 2 0 2 0 2 0 0 0 0]
  # [0 0 0 3] i don't care about this situation
  # [0 0 0 0 4]
  possibles = [math.inf] * (money+1)
  possibles[0] = 1
  # assuming its always possible as if there is always a 1 coin denomination
  for j in range(len(possibles)):
    if possibles[j]:
      for coin in coins:
        if j + coin <= money:
          possibles[j + coin] = min(possibles[j] + 1, possibles[j + coin])
  return possibles[money] - 1

In [2]:
print(dp_change(40,[50,25,20,10,5,1]))

2


In [3]:
dp_change(17028, [14,5,3,1]) # 1218

1218

In [72]:
def list_string_to_ints(s):
  return list(map(int, s.split()))

def read_manhattan(uri):
  content = read_uri(uri)
  n, m = list(map(int, content[0].split()))
  down = np.array(list(map(list_string_to_ints, content[1:n+1])))
  right = np.array(list(map(list_string_to_ints, content[n+2:2*n+3])))
  print(down, right)
  return n, m, down, right

In [73]:
read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/manhattan.txt")

[[1 0 2 4 3]
 [4 6 5 2 1]
 [4 4 5 2 1]
 [5 6 8 5 3]] [[3 2 4 0]
 [3 2 4 2]
 [0 7 3 3]
 [3 3 0 2]
 [1 3 2 2]]


(4, 4, array([[1, 0, 2, 4, 3],
        [4, 6, 5, 2, 1],
        [4, 4, 5, 2, 1],
        [5, 6, 8, 5, 3]]), array([[3, 2, 4, 0],
        [3, 2, 4, 2],
        [0, 7, 3, 3],
        [3, 3, 0, 2],
        [1, 3, 2, 2]]))

In [74]:
def longest_path(n, m, down, right):

  print(n, m, len(down), len(right))

  maximal = [0] * (m+1)
  for j in range(1, m + 1):
    maximal[j] = maximal[j - 1] + right[0][j - 1]

  print(maximal)

  for i in range(1, n+1):
    new_maximal = [0] * (m + 1)
    new_maximal[0] = maximal[0] + down[i - 1][0]
    for j in range(1, m+1):
      up = maximal[j] + down[i - 1][j]
      left = new_maximal[j-1] + right[i][j - 1]
      new_maximal[j] = max(up, left)
    maximal = new_maximal
  return maximal[m]

In [75]:
longest_path(*read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/manhattan.txt"))

[[1 0 2 4 3]
 [4 6 5 2 1]
 [4 4 5 2 1]
 [5 6 8 5 3]] [[3 2 4 0]
 [3 2 4 2]
 [0 7 3 3]
 [3 3 0 2]
 [1 3 2 2]]
4 4 4 5
[0, 3, 5, 9, 9]


34

In [76]:
longest_path(*read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/dataset_261_10.txt"))

[[4 3 3 1 3]
 [0 2 3 0 4]
 [3 1 2 2 4]
 [4 4 2 0 0]
 [1 2 1 4 0]
 [1 1 0 4 1]
 [1 0 1 2 0]
 [4 1 1 3 0]
 [4 2 0 1 3]
 [1 0 2 0 1]] [[3 4 0 0]
 [3 1 4 3]
 [4 1 1 1]
 [0 3 3 2]
 [4 4 1 0]
 [0 0 1 4]
 [3 0 2 3]
 [0 2 4 1]
 [3 1 4 3]
 [3 3 4 3]
 [2 1 4 2]]
10 4 10 11
[0, 3, 7, 7, 7]


40